In [10]:
import pathlib
import re
import json
from datetime import datetime
import flask
import dash
import dash_table
import matplotlib.colors as mcolors
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
# from precomputing import add_stopwords
from dash.dependencies import Output, Input, State
from dateutil import relativedelta
# from ldacomplaints import lda_analysis
from sklearn.manifold import TSNE
import sy_database
import sy_class

In [11]:
db_manager = sy_database.DatabaseManager()
category = db_manager.making_dataframe('category')
category

,제품카테고리,n,sum,재구매율,평균 재구매 주기(일)
0,Accessories,4,4,1.000000,13.541667
1,Android,1,1,1.000000,40.000000
2,Apparel,501,978,0.512270,3.517378
3,Backpacks,4,5,0.800000,12.357143
4,Bags,151,267,0.565543,16.477539
5,Bottles,24,32,0.750000,34.907692
6,Drinkware,246,436,0.564220,11.365633
7,Fun,13,19,0.684211,29.404762
8,Gift Cards,1,1,1.000000,0.837500
9,Google,5,8,0.625000,25.187500


In [12]:
import plotly.express as px

# Plotly Express를 사용하여 바 차트 그리기
def barplot1():
    # 기본 바 차트 생성
    fig = px.bar(category, 
                 x='평균 재구매 주기(일)',
                 y='제품카테고리', 
                 title='카테고리별 평균 재구매 주기(일)',
                 orientation='h',
                 color='제품카테고리',  # 카테고리에 따라 색상 지정
                 text='평균 재구매 주기(일)')  # 막대에 텍스트 라벨 추가
    
    # 차트 레이아웃 꾸미기
    fig.update_layout(
        title={
            'text': "카테고리별 평균 재구매 주기(일)",
            'y':0.9,
            'x':0.3,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="평균 재구매 주기(일)",
        yaxis_title="제품카테고리",
        legend_title="제품카테고리",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    
    # 막대 색상 꾸미기
    fig.update_traces(
        marker=dict(line=dict(color='#000000', width=1)),
        texttemplate='%{text:.2f}', textposition='outside'
    )
    
    return fig


barplot1()


In [13]:
import plotly.express as px

# Plotly Express를 사용하여 바 차트 그리기
def barplot2():
    # 기본 바 차트 생성
    fig = px.bar(category, 
                 x='재구매율',
                 y='제품카테고리',  
                 title='재구매율',
                 color='제품카테고리',  # 카테고리에 따라 색상 지정
                 orientation='h',
                 text='재구매율')  # 막대에 텍스트 라벨 추가
    
    # 차트 레이아웃 꾸미기
    fig.update_layout(
        title={
            'text': "재구매율",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="재구매율",
        yaxis_title="제품카테고리",
        legend_title="제품카테고리",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    
    # 막대 색상 꾸미기
    fig.update_traces(
        marker=dict(line=dict(color='#000000', width=1)),
        texttemplate='%{text:.2f}', textposition='outside'
    )
    
    return fig

# barplot1() 함수 호출하여 차트 출력
barplot2()


In [14]:
retention_matrix = db_manager.making_dataframe('retention_matrix')

In [15]:
import plotly.graph_objects as go
from datetime import datetime

def heatmap() : 
    fig = go.Figure(data=go.Heatmap(
        z=retention_matrix.values,
        x=retention_matrix.columns,
        y=retention_matrix.index,
        colorscale='Blues',
        text=retention_matrix.values,
        texttemplate="%{text:.2%}",  # 비율 형식으로 표시
        textfont={"size": 10, "color": "black"}
    ))
    
    # 레이아웃 업데이트
    fig.update_layout(
        title='코호트 분석 - Retention Rates',
        title_font_size=20,
        xaxis_title='Months After First Purchase',
        yaxis_title='Cohort Group',
        xaxis=dict(tickmode='linear', tick0=0, dtick=1, tickfont=dict(size=12)),
        yaxis=dict(tickfont=dict(size=12)),
        plot_bgcolor='white',
        margin=dict(l=60, r=20, t=50, b=50),
        width=800,
        height=600
    )
    
    # 축 레이블 및 타이틀 스타일링
    fig.update_xaxes(
        title_font=dict(size=14),
        tickfont=dict(size=12)
    )
    
    fig.update_yaxes(
        title_font=dict(size=14),
        tickfont=dict(size=12)
    )
    
    # 색상바(컬러바) 스타일링
    fig.update_coloraxes(colorbar=dict(
        title="재구매율",
        titleside="right",
        titlefont=dict(size=14),
        tickfont=dict(size=12)
    ))
    return fig

heatmap()

In [16]:
# NAVBAR 구성
NAVBAR = dbc.Navbar(
    children=[
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(),
                    dbc.Col(
                        dbc.NavbarBrand("Bank Customer Complaints", className="ml-2")
                    ),
                ],
                align="center",
            ),
            href="https://plot.ly",
        )
    ],
    color="dark",
    dark=True,
    sticky="top",
)

CLUSTER1 = dbc.Card(
    [
        dbc.CardHeader(html.H5("Comparison of each cluster")),
        dbc.CardBody(
            [
                dcc.Loading(
                    id="loading-cluster-comps",
                    children=[
                        dbc.Alert(
                            "Something's gone wrong! Give us a moment, but try loading this page again if problem persists.",
                            id="cluster-alert",  # ID 변경
                            color="warning",
                            style={"display": "none"},
                        ),
                        dcc.Graph(id="cluster-comps", figure=barplot1()),   
                    ],
                    type="default",
                )
            ],
            style={"marginTop": 0, "marginBottom": 0},
        ),
    ]
)

CLUSTER2 = dbc.Card(
    [
        dbc.CardHeader(html.H5("Comparison of each cluster")),
        dbc.CardBody(
            [
                dcc.Loading(
                    id="loading-cluster-comps",
                    children=[
                        dbc.Alert(
                            "Something's gone wrong! Give us a moment, but try loading this page again if problem persists.",
                            id="cluster-alert",  # ID 변경
                            color="warning",
                            style={"display": "none"},
                        ),
                        dcc.Graph(id="cluster-comps", figure=barplot2()),  
                    ],
                    type="default",
                )
            ],
            style={"marginTop": 0, "marginBottom": 0},
        ),
    ]
)

CLUSTER3 = dbc.Card(
    [
        dbc.CardHeader(html.H5("Comparison of each cluster")),
        dbc.CardBody(
            [
                dcc.Loading(
                    id="loading-cluster-comps",
                    children=[
                        dbc.Alert(
                            "Something's gone wrong! Give us a moment, but try loading this page again if problem persists.",
                            id="cluster-alert",  # ID 변경
                            color="warning",
                            style={"display": "none"},
                        ),
                        dcc.Graph(id="cluster-comps", figure=heatmap()),  
                    ],
                    type="default",
                )
            ],
            style={"marginTop": 0, "marginBottom": 0},
        ),
    ]
)


In [17]:
# 전체 레이아웃 구성 변경
BODY = dbc.Container(
    [
        dbc.Row(
            [   
                dbc.Col(CLUSTER1, align="center"),
                dbc.Col(CLUSTER2, align="center")
            
            ],
            
            style={"marginTop": 30}
        ),
        dbc.Row(
            [   
                dbc.Col(CLUSTER3, align="center"),
            
            ],
            
            style={"marginTop": 30}
        ),
        
    ],
    className="mt-12",
)

In [18]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server  # for Heroku deployment

app.layout = html.Div(children=[NAVBAR, BODY])

if __name__ == "__main__":
    app.run_server(jupyter_mode='external')

[2024-06-11 14:50:20,994] ERROR in app: Exception on /_alive_f087de49-0779-4354-9eeb-3ac59ee15fb7 [GET]
Traceback (most recent call last):
  File "/Users/yang/Library/Python/3.9/lib/python/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/yang/Library/Python/3.9/lib/python/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/yang/Library/Python/3.9/lib/python/site-packages/flask/app.py", line 878, in full_dispatch_request
    rv = self.preprocess_request()
  File "/Users/yang/Library/Python/3.9/lib/python/site-packages/flask/app.py", line 1253, in preprocess_request
    rv = self.ensure_sync(before_func)()
  File "/Users/yang/Library/Python/3.9/lib/python/site-packages/dash/dash.py", line 1412, in _setup_server
    _validate.validate_layout(self.layout, self._layout_value())
  File "/Users/yang/Library/Python/3.9/lib/python/site-packages/dash/_validate.py", line

Dash app running on http://127.0.0.1:8050/
